<a href="https://colab.research.google.com/github/royam0820/nanochat-speedrun-001/blob/main/nanochat_CPU_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NanoChat Text Generation (CPU-Compatible)

This notebook runs the standalone NanoChat model for text generation. No additional installations beyond the listed dependencies are needed.

**Model Size:** ~2GB (downloads once to `/content/nanochat` in Colab's temp storage).  
**Runtime:** CPU (editable in Runtime > Change runtime type).  
**Dependencies:** torch, tiktoken, numpy, huggingface_hub.

Run the cells below in order.

In [1]:
# Install required packages
!pip install torch tiktoken numpy huggingface_hub

print("Dependencies installed!")

Dependencies installed!


## Download Model Files

Downloads the NanoChat model from Hugging Face if not already present. This may take 5-10 minutes depending on your connection.

In [2]:
import os
from huggingface_hub import snapshot_download

model_dir = "/content/nanochat"  # Colab's persistent temp dir
os.makedirs(model_dir, exist_ok=True)

# Check if model is already downloaded (rough check: look for main model file)
model_file = os.path.join(model_dir, "model_000650.pt")
if os.path.exists(model_file) and os.path.getsize(model_file) > 1000000:  # >1MB
    print("Model files already downloaded, skipping.")
else:
    print("Downloading NanoChat model files from HuggingFace...")
    print("This may take a while (model is ~2GB)...")
    snapshot_download(
        repo_id="sdobson/nanochat",
        #repo_id="royam0820/nanochat-speedrun-001",
        local_dir=model_dir,
        local_dir_use_symlinks=False,
    )
    print(f"Model downloaded successfully to {model_dir}")

This may take a while (model is ~2GB)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

meta_000650.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.pkl:   0%|          | 0.00/846k [00:00<?, ?B/s]

model_000650.pt:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

token_bytes.pt:   0%|          | 0.00/264k [00:00<?, ?B/s]

Model downloaded successfully to /content/nanochat


## Model and Generation Code

Below is the adapted generation code from `generate_cpu.py`. It defines the minimal GPT model, loads weights, and generates text.

Customize the prompt, max tokens, temperature, and top-k in the next cell.

Code (Define Model and Generation Function)

In [3]:
import argparse
import torch
import os
import json
import glob
import pickle
import math
from dataclasses import dataclass

import torch.nn as nn
import torch.nn.functional as F

# Minimal GPT implementation (adapted from nanochat)
@dataclass
class GPTConfig:
    sequence_len: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 6
    n_kv_head: int = 6
    n_embd: int = 768

def norm(x):
    return F.rms_norm(x, (x.size(-1),))

def apply_rotary_emb(x, cos, sin):
    assert x.ndim == 4
    d = x.shape[3] // 2
    x1, x2 = x[..., :d], x[..., d:]
    y1 = x1 * cos + x2 * sin
    y2 = x1 * (-sin) + x2 * cos
    out = torch.cat([y1, y2], 3)
    out = out.to(x.dtype)
    return out

def repeat_kv(x, n_rep):
    if n_rep == 1:
        return x
    bs, n_kv_heads, slen, head_dim = x.shape
    return (
        x[:, :, None, :, :]
        .expand(bs, n_kv_heads, n_rep, slen, head_dim)
        .reshape(bs, n_kv_heads * n_rep, slen, head_dim)
    )

class CausalSelfAttention(nn.Module):
    def __init__(self, config, layer_idx):
        super().__init__()
        self.layer_idx = layer_idx
        self.n_head = config.n_head
        self.n_kv_head = config.n_kv_head
        self.n_embd = config.n_embd
        self.head_dim = self.n_embd // self.n_head
        assert self.n_embd % self.n_head == 0
        assert self.n_kv_head <= self.n_head and self.n_head % self.n_kv_head == 0
        self.c_q = nn.Linear(self.n_embd, self.n_head * self.head_dim, bias=False)
        self.c_k = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)
        self.c_v = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)
        self.c_proj = nn.Linear(self.n_embd, self.n_embd, bias=False)

    def forward(self, x, cos_sin, kv_cache):
        B, T, C = x.size()
        q = self.c_q(x).view(B, T, self.n_head, self.head_dim)
        k = self.c_k(x).view(B, T, self.n_kv_head, self.head_dim)
        v = self.c_v(x).view(B, T, self.n_kv_head, self.head_dim)
        cos, sin = cos_sin
        q, k = apply_rotary_emb(q, cos, sin), apply_rotary_emb(k, cos, sin)
        q, k = norm(q), norm(k)
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
        if kv_cache is not None:
            k, v = kv_cache.insert_kv(self.layer_idx, k, v)
        Tq = q.size(2)
        Tk = k.size(2)
        nrep = self.n_head // self.n_kv_head
        k, v = repeat_kv(k, nrep), repeat_kv(v, nrep)
        if kv_cache is None or Tq == Tk:
            y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        elif Tq == 1:
            y = F.scaled_dot_product_attention(q, k, v, is_causal=False)
        else:
            attn_mask = torch.zeros((Tq, Tk), dtype=torch.bool, device=q.device)
            prefix_len = Tk - Tq
            if prefix_len > 0:
                attn_mask[:, :prefix_len] = True
            attn_mask[:, prefix_len:] = torch.tril(torch.ones((Tq, Tq), dtype=torch.bool, device=q.device))
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask)
        y = y.transpose(1, 2).contiguous().view(B, T, -1)
        y = self.c_proj(y)
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=False)
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=False)

    def forward(self, x):
        x = self.c_fc(x)
        x = F.relu(x).square()
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config, layer_idx):
        super().__init__()
        self.attn = CausalSelfAttention(config, layer_idx)
        self.mlp = MLP(config)

    def forward(self, x, cos_sin, kv_cache):
        x = x + self.attn(norm(x), cos_sin, kv_cache)
        x = x + self.mlp(norm(x))
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict({
            "wte": nn.Embedding(config.vocab_size, config.n_embd),
            "h": nn.ModuleList([Block(config, layer_idx) for layer_idx in range(config.n_layer)]),
        })
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.rotary_seq_len = config.sequence_len * 10
        head_dim = config.n_embd // config.n_head
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)
        self.register_buffer("cos", cos, persistent=False)
        self.register_buffer("sin", sin, persistent=False)
        self.transformer.wte.to(dtype=torch.bfloat16)

    def init_weights(self):
        self.apply(self._init_weights)
        torch.nn.init.zeros_(self.lm_head.weight)
        for block in self.transformer.h:
            torch.nn.init.zeros_(block.mlp.c_proj.weight)
            torch.nn.init.zeros_(block.attn.c_proj.weight)
        head_dim = self.config.n_embd // self.config.n_head
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)
        self.cos, self.sin = cos, sin

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            fan_out = module.weight.size(0)
            fan_in = module.weight.size(1)
            std = 1.0 / math.sqrt(fan_in) * min(1.0, math.sqrt(fan_out / fan_in))
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0)

    def _precompute_rotary_embeddings(self, seq_len, head_dim, base=10000, device=None):
        if device is None:
            device = self.transformer.wte.weight.device
        channel_range = torch.arange(0, head_dim, 2, dtype=torch.float32, device=device)
        inv_freq = 1.0 / (base ** (channel_range / head_dim))
        t = torch.arange(seq_len, dtype=torch.float32, device=device)
        freqs = torch.outer(t, inv_freq)
        cos, sin = freqs.cos(), freqs.sin()
        cos, sin = cos.bfloat16(), sin.bfloat16()
        cos, sin = cos[None, :, None, :], sin[None, :, None, :]
        return cos, sin

    def forward(self, idx, targets=None, kv_cache=None):
        B, T = idx.size()
        assert T <= self.cos.size(1)
        T0 = 0 if kv_cache is None else kv_cache.get_pos()
        cos_sin = self.cos[:, T0:T0+T], self.sin[:, T0:T0+T]
        x = self.transformer.wte(idx)
        x = norm(x)
        for block in self.transformer.h:
            x = block(x, cos_sin, kv_cache)
        x = norm(x)
        softcap = 15
        logits = self.lm_head(x)
        logits = softcap * torch.tanh(logits / softcap)
        return logits

# Function to load and generate (adapted from main script)
def generate_text(model_dir, prompt="Once upon a time", max_tokens=100, temperature=0.8, top_k=50, device="cpu"):
    device = torch.device(device)
    print(f"Using device: {device}")

    # Find model and meta files
    model_file = os.path.join(model_dir, "model_000650.pt") # Correctly reference the model file name
    if not os.path.exists(model_file):
        raise FileNotFoundError(f"Model file '{model_file}' not found.")

    meta_files = glob.glob(os.path.join(model_dir, "meta_*.json"))
    if not meta_files:
        raise FileNotFoundError(f"No meta files found in {model_dir}")
    meta_file = meta_files[0]

    print(f"Loading model from {model_file}")
    print(f"Loading metadata from {meta_file}")

    # Load metadata
    with open(meta_file, 'r') as f:
        meta = json.load(f)

    model_config_kwargs = meta["model_config"]
    print(f"Model config: {model_config_kwargs}")

    # Build the model
    model_config = GPTConfig(**model_config_kwargs)
    with torch.device("meta"):
        model = GPT(model_config)

    # Load weights
    print("Loading model weights (this may take a minute for a 2GB model)...")
    model_data = torch.load(model_file, map_location=device, weights_only=False)
    model_data = {k.lstrip("_orig_mod."): v for k, v in model_data.items()}

    # Convert to float32 for CPU
    print("Converting model to float32 for CPU...")
    model_data = {k: v.float() if v.dtype == torch.bfloat16 else v for k, v in model_data.items()}

    model.to_empty(device=device)
    model.init_weights()
    model.load_state_dict(model_data, strict=True, assign=True)
    model.eval()
    print("Model loaded successfully!")

    # Load tokenizer
    print("Loading tokenizer...")
    tokenizer_path = os.path.join(model_dir, "tokenizer.pkl")
    if not os.path.exists(tokenizer_path):
        raise FileNotFoundError(f"Tokenizer not found at {tokenizer_path}.")

    with open(tokenizer_path, "rb") as f:
        import tiktoken
        enc = pickle.load(f)

    # BOS token
    try:
        bos_token_id = enc.encode_single_token("<|bos|>")
    except KeyError:
        try:
            bos_token_id = enc.encode_single_token("<|endoftext|>")
        except:
            bos_token_id = None

    print("Tokenizer loaded successfully!")

    # Encode prompt
    input_ids = enc.encode_ordinary(prompt)
    print(f"\nPrompt: {prompt}")
    print(f"Encoded to {len(input_ids)} tokens")
    print()

    # Generate
    print("Generating...")
    print("-" * 50)
    print(prompt, end="", flush=True)

    x = torch.tensor([input_ids], dtype=torch.long, device=device)

    with torch.inference_mode():
        for _ in range(max_tokens):
            logits = model(x)
            logits = logits[:, -1, :]

            logits = logits / temperature

            if top_k > 0:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

            token_str = enc.decode([next_token.item()])
            print(token_str, end="", flush=True)

            x = torch.cat([x, next_token], dim=1)

            if bos_token_id is not None and next_token.item() == bos_token_id:
                break

    print()
    print("-" * 50)
    print("\nGeneration complete!")

print("Model and generation function defined!")

Model and generation function defined!


## Run Generation

Customize parameters below and run. Example prompt: "Tell me about dogs."

In [4]:
# Customize these parameters
MODEL_DIR = "/content/nanochat"
PROMPT = "Tell me about dogs."  # Change this!
MAX_TOKENS = 100
TEMPERATURE = 0.8
TOP_K = 50
DEVICE = "cpu"  # Or "cuda" if using GPU

# Generate!
generate_text(
    model_dir=MODEL_DIR,
    prompt=PROMPT,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    top_k=TOP_K,
    device=DEVICE
)

Using device: cpu
Loading model from /content/nanochat/model_000650.pt
Loading metadata from /content/nanochat/meta_000650.json
Model config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 20, 'n_head': 10, 'n_kv_head': 10, 'n_embd': 1280}
Loading model weights (this may take a minute for a 2GB model)...
Converting model to float32 for CPU...
Model loaded successfully!
Loading tokenizer...
Tokenizer loaded successfully!

Prompt: Tell me about dogs.
Encoded to 5 tokens

Generating...
--------------------------------------------------
Tell me about dogs. That is something I used to do in my childhood. Now I am very old. I am a doctor. I work in a hospital. I also do a lot of things for children. I am very fond of birds and I am always looking for ways to help them. I have got a lot of birds in my collection. It's said that I might have five or six birds now. I like birds very much. What is the best title for this passage?   _  .
- A
-----------------------------------------------

In [ ]:
# ============================================
# 📊 Visualize Tokenization with Colors
# ============================================

# Reload the tokenizer (already loaded in generate_text function)
import pickle
import tiktoken

MODEL_DIR = "/content/nanochat"

print("Loading tokenizer for visualization...")
tokenizer_path = f"{MODEL_DIR}/tokenizer.pkl"

with open(tokenizer_path, "rb") as f:
    enc = pickle.load(f)

print("✅ Tokenizer loaded!\n")

# Define visualization function (adapted from tokenizer.py)
def visualize_tokenization(enc, ids, mask, with_token_id=False):
    """Visualize tokenization with colors"""
    RED = '\033[91m'
    GREEN = '\033[92m'
    RESET = '\033[0m'
    GRAY = '\033[90m'

    tokens = []
    for i, (token_id, mask_val) in enumerate(zip(ids, mask)):
        token_str = enc.decode([token_id])
        color = GREEN if mask_val == 1 else RED
        tokens.append(f"{color}{token_str}{RESET}")
        if with_token_id:
            tokens.append(f"{GRAY}({token_id}){RESET}")

    return '|'.join(tokens)

# Example 1: Visualize simple text
print("="*60)
print("EXAMPLE 1: Simple text tokenization")
print("="*60)

text = "Tell me about dogs."
ids = enc.encode_ordinary(text)
mask = [1] * len(ids)  # All tokens supervised (green)

print(f"\nOriginal text: {text}")
print(f"Number of tokens: {len(ids)}")
print(f"Token IDs: {ids}")
print("\nVisualization (🟢 GREEN=supervised, 🔴 RED=not supervised):")
print(visualize_tokenization(enc, ids, mask))

print("\n\nWith token IDs:")
print(visualize_tokenization(enc, ids, mask, with_token_id=True))

# Example 2: Simulate a conversation structure
print("\n" + "="*60)
print("EXAMPLE 2: Conversation-style tokenization")
print("="*60)

# Get special tokens
try:
    bos = enc.encode_single_token("<|bos|>")
    user_start = enc.encode_single_token("<|user_start|>")
    user_end = enc.encode_single_token("<|user_end|>")
    assistant_start = enc.encode_single_token("<|assistant_start|>")
    assistant_end = enc.encode_single_token("<|assistant_end|>")

    # Build conversation manually
    user_text = "What is 2 + 2?"
    assistant_text = "The answer is 4."

    user_ids = enc.encode_ordinary(user_text)
    assistant_ids = enc.encode_ordinary(assistant_text)

    # Construct full conversation
    ids = [bos] + [user_start] + user_ids + [user_end] + [assistant_start] + assistant_ids + [assistant_end]

    # Mask: 0 for user/special tokens, 1 for assistant content
    mask = ([0] + [0] + [0]*len(user_ids) + [0] + [0] + [1]*len(assistant_ids) + [1])

    print(f"\nUser: {user_text}")
    print(f"Assistant: {assistant_text}")
    print(f"\nTotal tokens: {len(ids)}")
    print(f"Supervised tokens (green): {sum(mask)}")
    print(f"Non-supervised tokens (red): {len(mask) - sum(mask)}")

    print("\nVisualization:")
    print(visualize_tokenization(enc, ids, mask))

except KeyError:
    print("⚠️  Special tokens not found in this tokenizer. Showing simple version:")

    user_text = "User: What is 2 + 2?"
    assistant_text = " Assistant: The answer is 4."

    user_ids = enc.encode_ordinary(user_text)
    assistant_ids = enc.encode_ordinary(assistant_text)

    ids = user_ids + assistant_ids
    mask = [0]*len(user_ids) + [1]*len(assistant_ids)

    print(f"\nTotal tokens: {len(ids)}")
    print(f"User tokens (red): {len(user_ids)}")
    print(f"Assistant tokens (green): {len(assistant_ids)}")

    print("\nVisualization:")
    print(visualize_tokenization(enc, ids, mask))

# Example 3: Visualize your custom prompt
print("\n" + "="*60)
print("EXAMPLE 3: Your generation prompt")
print("="*60)

custom_prompt = PROMPT  # From the generation cell
ids = enc.encode_ordinary(custom_prompt)
mask = [0] * len(ids)  # User input = not supervised (red)

print(f"\nYour prompt: {custom_prompt}")
print(f"Number of tokens: {len(ids)}")
print(f"Token IDs: {ids}")

print("\nVisualization (all red = user input):")
print(visualize_tokenization(enc, ids, mask))

# Example 4: Compare different texts
print("\n" + "="*60)
print("EXAMPLE 4: Compare number tokenization")
print("="*60)

test_texts = [
    "123",
    "1234",
    "The year 2024",
    "Price: $99.99"
]

for text in test_texts:
    ids = enc.encode_ordinary(text)
    mask = [1] * len(ids)
    print(f"\nText: '{text}' → {len(ids)} tokens")
    print(visualize_tokenization(enc, ids, mask, with_token_id=True))

print("\n✅ Tokenization visualization complete!")

Loading tokenizer for visualization...
✅ Tokenizer loaded!

EXAMPLE 1: Simple text tokenization

Original text: Tell me about dogs.
Number of tokens: 5
Token IDs: [26256, 551, 567, 4090, 46]

Visualization (🟢 GREEN=supervised, 🔴 RED=not supervised):
Tell| me| about| dogs|.


With token IDs:
Tell|(26256)| me|(551)| about|(567)| dogs|(4090)|.|(46)

EXAMPLE 2: Conversation-style tokenization

User: What is 2 + 2?
Assistant: The answer is 4.

Total tokens: 19
Supervised tokens (green): 7
Non-supervised tokens (red): 12

Visualization:
<|bos|>|<|user_start|>|What| is| |2| +| |2|?|<|user_end|>|<|assistant_start|>|The| answer| is| |4|.|<|assistant_end|>

EXAMPLE 3: Your generation prompt

Your prompt: Tell me about dogs.
Number of tokens: 5
Token IDs: [26256, 551, 567, 4090, 46]

Visualization (all red = user input):
Tell| me| about| dogs|.

EXAMPLE 4: Compare number tokenization

Text: '123' → 2 tokens
12|(1036)|3|(51)

Text: '1234' → 2 tokens
12|(1036)|34|(4196)

Text: 'The year 2024' → 5 t

## Notes
- **Re-running:** Model download and loading happen only once per session (Colab resets on disconnect).
- **GPU Acceleration:** Change `DEVICE = "cuda"` and runtime type for faster inference (script auto-converts weights).
- **Save Output:** Use File > Download .ipynb to save the notebook.
- **Original Files:** This adapts `download_model.py`, `launch.sh`, and `generate_cpu.py` into Colab cells.

Enjoy generating with NanoChat!